# FIT5196 Assessment 1
#### Student Name: Jiawei Ren
#### Student ID: 32073119

## Task 2: Parsing Text Files
<br> 1. Generate the corpus vocabulary
<br> 2. generate the sparse representation 


## Table of Contents

* [1. Import Library](#task1)
* [2. read excel files and stopwords from current working directory](#task2)
* [3. Read data into one dataframe and change the format of reviewTime](#task3)
* [4. Tokenization， case normalization and removing stop words for string in reviewText and summary](#task4)
* [5. Count word frequency and find rare words](#task5)
* [6. Find context dependent stopwords](#task6)
* [7. Apply Porterstemmer to words in dataframe](#task7)
* [8. Combine unigrams and bigrams,then sort alphabetically](#task8)
* [9. create countvectorizer](#task9)

## Step 1 Import Library: <a class="anchor" id="task1"></a>

In [1]:
import nltk 
import pandas as pd # for manage excel files and dataframe
import os  #for removing files
from nltk.corpus import stopwords #for removing stop words and create vocabulary
from nltk.tokenize import RegexpTokenizer # for creating tokenizer and tokens
from nltk.stem import PorterStemmer #for performing porterstemmer
from nltk.probability import * #count frequency of word
from sklearn.feature_extraction.text import CountVectorizer #for creating count vectorizer 
from nltk.util import ngrams #for creating n-grams
from nltk.tokenize import MWETokenizer # for multi-word expressions.

## Step 2 read excel files and stopwords from current working directory <a class="anchor" id="task2"></a>

In [2]:
# read unstructured EXCEL files
df1 = pd.read_excel("32073119.xlsx", sheet_name = 0, header = 0, usecols = "J:M") 
df2 = pd.read_excel("32073119.xlsx", sheet_name = 1, header = 7, usecols = "B:D") 
df3 = pd.read_excel("32073119.xlsx", sheet_name = 2, header = 9, usecols = "G:I") 
df4 = pd.read_excel("32073119.xlsx", sheet_name = 3, header = 6, usecols = "A:C") 
df5 = pd.read_excel("32073119.xlsx", sheet_name = 4, header = 7, usecols = "B:D") 
df6 = pd.read_excel("32073119.xlsx", sheet_name = 5, header = 1, usecols = "I:K") 
df7 = pd.read_excel("32073119.xlsx", sheet_name = 6, header = 2, usecols = "C:E") 
df8 = pd.read_excel("32073119.xlsx", sheet_name = 7, header = 9, usecols = "E:G") 
df9 = pd.read_excel("32073119.xlsx", sheet_name = 8, header = 4, usecols = "E:G") 
df10 = pd.read_excel("32073119.xlsx", sheet_name = 9, header = 7, usecols = "D:F") 
df11 = pd.read_excel("32073119.xlsx", sheet_name = 10, header = 2, usecols = "D:F") 
df12 = pd.read_excel("32073119.xlsx", sheet_name = 11, header = 0, usecols = "C:E") 
df13 = pd.read_excel("32073119.xlsx", sheet_name = 12, header = 9, usecols = "G:I") 
df14 = pd.read_excel("32073119.xlsx", sheet_name = 13, header = 5, usecols = "J:L") 
df15 = pd.read_excel("32073119.xlsx", sheet_name = 14, header = 9, usecols = "D:F") 
df16 = pd.read_excel("32073119.xlsx", sheet_name = 15, header = 4, usecols = "C:E") 
df17 = pd.read_excel("32073119.xlsx", sheet_name = 16, header = 9, usecols = "H:J") 
df18 = pd.read_excel("32073119.xlsx", sheet_name = 17, header = 6, usecols = "F:H") 
df19 = pd.read_excel("32073119.xlsx", sheet_name = 18, header = 7, usecols = "D:F") 
df20 = pd.read_excel("32073119.xlsx", sheet_name = 19, header = 4, usecols = "H:J") 
#read stopwords
stopwords = []
with open('stopwords_en.txt') as f:
    for word in f:
        stopwords.append(word.strip())
f.close()

## Step 3 Read data into one dataframe and change the format of reviewTime <a class="anchor" id="task3"></a>

In [3]:
#set the length of column, so I can see the full text
pd.options.display.max_colwidth = 100000
#make a concat list
df_list = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20]
appended_data = pd.concat(df_list)
# create a combined EXCEL file and read it 
appended_data.to_excel('appended.xlsx')
df = pd.read_excel('appended.xlsx', header = 0, usecols = "B:D") 
#convert reviewTime to datetime format
df['reviewTime'] = pd.to_datetime(df['reviewTime'])
df['reviewTime'] = df['reviewTime'].dt.strftime('%d/%m/%Y')
print("full dataframe \n")
df.head()

full dataframe 



,reviewText,summary,reviewTime
0,"This down-to-earth, matter-of-fact book will either bring to remembrance the rules of punctuation you once knew, or, if you never learned them, there's enough information to learn a whole new set of skills. It's not only helpful, but also entertaining. I enjoyed reading it, and found it to be helpful.",Outstanding,08/03/2014
1,Man I need part 5 like right now.. I love your pen game your a beast at what you do. I need Rylan and Iman to become one 4real,LUVED IT !!!!!,24/06/2014
2,"My grandson had filled this in within two days of receipt. It was fun, and he loved it, but I do with it had had more pages. He loves dot-to-dot!",This Didn't Last Long,15/10/2013
3,"Her Place in Time is my 500th Kindle book since I began reading e-books in 2010. Her Place in Time is one of the rare books I would rate at 6+ Stars! Yes, it's that good.However, in order to enjoy this book, you need to be a reader that appreciates a blend of high-energy action adventure spiced with some steamy romance. I adore well-written time travel stories and this book exceeded my expectations. In fact, Ms. Jackson does such exceptional world building in her novel that I felt I was actually seeing a movie with great dialogue! Sights, sounds, smells were so wonderfully portrayed that I felt I was actually in the middle of all the action. When the h swung her blade, I swung a blade and when she fought the assassins, I was fighting right along side with her....Yes, this book is for all of us who can cheer a smart, strong self-sufficient female warrior handling herself in any situation (while still maintaining her humanity) and flinch at the typical simpering, whimpering, helpless, sweet female heroines.I was hooked from the beginning of the book to the very last page. I won't give you any spoilers but the story is about a highly trained black ops agent who after an emotionally devastating mission goes into hiding from her own agency who have sent assassins to eliminate her. She finds a way to go back in time to the early Viking period where her warrior skills are highly valued.The Kindle price is a bit more than I normally pay but it was so worth it. I also purchased Ms. Jackson's other time travel, Timeless Knights, which I equally devoured with pleasure!I truly hope in the near future that Ms. Jackson gives David his own time travel novel. Ms. Jackson, please focus your energies in this genre as there are so few well written time travel novels available.",Uber Strong Heroine,13/01/2013
4,"While not my favorite of the trilogy, this waw tied with the second installment. Both characters were lovable and you wanted them to win from the moment they began working as pairs against all odds.",Pair Skating with a flair,20/07/2008


## Step 4 Tokenization， case normalization and removing stop words for string in reviewText and summary  <a class="anchor" id="task4"></a>

In [4]:
# create tokenizer pattern
regexptokenizer = RegexpTokenizer("[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
dftoken = df
vocab = []
text_vocab = []
for i in range(10000):
    #tokenization
    filtered_reviewText = []
    filtered_summary = []
    words1 = regexptokenizer.tokenize(str(dftoken["reviewText"][i]))
    words2 = regexptokenizer.tokenize(str(dftoken["summary"][i]))
    #case normalization
    for x in range(len(words1)):
        words1[x] = words1[x].lower()
    for x in range(len(words2)):
        words2[x] = words2[x].lower()
    #remove context-independent stopwords and token with a length less than 3
    for word in words1:
        if word not in stopwords and len(word) >= 3:
            filtered_reviewText.append(word)
    for word in words2:
        if word not in stopwords and len(word) >= 3:  
            filtered_summary.append(word)
    #replace string with token list
    dftoken.at[i, 'reviewText'] = filtered_reviewText
    dftoken.at[i, 'summary'] = filtered_summary
    #store all tokens in a list
    vocab = vocab + filtered_summary + filtered_reviewText
#use set to get a unique vocabulary
vocab = set(vocab)
print("tokenized dataframe \n")
dftoken.head()

tokenized dataframe 



,reviewText,summary,reviewTime
0,"[down-to, earth, matter-of, fact, book, bring, remembrance, rules, punctuation, knew, learned, information, learn, set, skills, helpful, entertaining, enjoyed, reading, found, helpful]",[outstanding],08/03/2014
1,"[man, part, love, pen, game, beast, rylan, iman, real]",[luved],24/06/2014
2,"[grandson, filled, days, receipt, fun, loved, pages, loves, dot-to, dot]",[long],15/10/2013
3,"[place, time, kindle, book, began, reading, e-books, place, time, rare, books, rate, stars, good, order, enjoy, book, reader, appreciates, blend, high-energy, action, adventure, spiced, steamy, romance, adore, well-written, time, travel, stories, book, exceeded, expectations, fact, jackson, exceptional, world, building, felt, movie, great, dialogue, sights, sounds, smells, wonderfully, portrayed, felt, middle, action, swung, blade, swung, blade, fought, assassins, fighting, side, book, cheer, smart, strong, self-sufficient, female, warrior, handling, situation, maintaining, humanity, flinch, typical, simpering, whimpering, helpless, sweet, female, heroines, hooked, beginning, book, page, give, spoilers, story, highly, trained, black, ops, agent, emotionally, devastating, mission, hiding, agency, assassins, eliminate, finds, back, time, ...]","[uber, strong, heroine]",13/01/2013
4,"[favorite, trilogy, waw, tied, installment, characters, lovable, wanted, win, moment, began, working, pairs, odds]","[pair, skating, flair]",20/07/2008


## Step 5 Count word frequency and find rare words  <a class="anchor" id="task5"></a>

In [5]:
unique_date = []
date_word= []
word_count ={}
#create a list of unique date
for date in dftoken["reviewTime"]:
    if date not in unique_date:
        unique_date.append(date)
# count word by date
for date in unique_date:
    dword = []
    for texts in dftoken["reviewText"].loc[dftoken['reviewTime'] == date]:
        for text in texts:
            dword.append(text)
            #record each word in the dictionary
            word_count[text] = 0
    for summary in dftoken["summary"].loc[dftoken['reviewTime'] == date]:
        for text in summary:
            dword.append(text)
            word_count[text] = 0
    # this list is like [a date, and all unique tokens at this day]
    date_word.append([date,set(dword)])
    
for i in range(len(date_word)):
    # count the frequency of words
    for word in date_word[i][1]:
        word_count[word] +=1
rare_word = []
print("list of rare word \n")
for (key, value) in word_count.items():
    if value <10:
        rare_word.append(key)

list of rare word 



In [6]:
print("I found {} rare words".format(len(rare_word)))

I found 40130 rare words


In [7]:
# remove rare word from vocabulary and dictionary
for word in rare_word:
    vocab.remove(word)
    word_count.pop(word, None)

In [8]:
print("Now I have {} words in vocabulary".format(len(vocab)))

Now I have 6270 words in vocabulary


## Step 6 Find context dependent stopwords  <a class="anchor" id="task6"></a>

In [9]:
context_dependent = []
for row in range(10000):
    # count the frequency of words in a cell
    f_summary = FreqDist(dftoken["summary"][row])
    f_review = FreqDist(dftoken["reviewText"][row])
    for text in dftoken["reviewText"][row]:
        # pass if the frequency is greater than Number_of_days / 2
        if text in word_count and f_review[text] > word_count[text]/2:
            context_dependent.append(text)
    for text in dftoken["summary"][row]:
        if text in word_count and f_summary[text] > word_count[text]/2:
            context_dependent.append(text)
#get a unique list of context dependent words
context_dependent = set(context_dependent)
print(context_dependent)
print("\n\nnumber of context_dependent words found:", len(context_dependent))

{'natalie', 'woody', 'communist', 'jesse', 'taboo', 'emma', 'adam', 'antennas', 'cia', 'cooker', 'arthur', 'brandon', 'aaron', 'stones', 'restoration', 'kong', 'linux', 'raven', 'physics', 'finder', 'roku', 'networking', 'iso', 'hannah', 'josh', 'walter', 'compartment', 'carol', 'musicals', 'pearl', 'sophie', 'leap', 'cole', 'theories', 'camcorder', 'cheese', 'savage', 'todd', 'spring', 'batman', 'jeans', 'ally', 'india', 'collins', 'lucy', 'hardy', 'kristen', 'claire', 'maggie', 'norman', 'texture', 'tommy', 'letters', 'guest', 'philosophical', 'bose', 'seattle', 'pioneer', 'superman', 'liam', 'lex', 'sata', 'reed', 'quantum', 'holocaust', 'harper', 'wwe', 'earphones', 'clay', 'kyle', 'ross', 'painting', 'dick', 'seth', 'dylan', 'cloud', 'raid', 'trips', 'lily', 'bout', 'walt', 'poems', 'nathan', 'boom', 'matrix', 'ellen', 'meal', 'moore', 'andrew', 'converter', 'bailey', 'scripture', 'cruise', 'bandwidth', 'ash', 'criterion', 'sally', 'ring', 'mars', 'maximum', 'june', 'matthew', 'va

In [10]:
# remove context dependent words from vocabulary and dictionary 
for word in context_dependent:
    vocab.remove(word)
    word_count.pop(word, None)

In [11]:
print("number of unique words now:", len(vocab))

number of unique words now: 6161


## Step 7 Apply Porterstemmer to words in dataframe  <a class="anchor" id="task7"></a>

In [12]:
pstemmer = PorterStemmer()
all_tokens = []
date_token = []
#Apply Porterstemmer to words in dataframe
for row in range(10000):
    filtered_reviewText = []
    filtered_summary = []
    for text in dftoken["reviewText"][row]:
        if text in vocab:
            filtered_reviewText.append(pstemmer.stem(text))
    for text in dftoken["summary"][row]:
        if text in vocab:
            filtered_summary.append(pstemmer.stem(text))
    dftoken.at[row, 'reviewText'] = filtered_reviewText
    dftoken.at[row, 'summary'] = filtered_summary
    all_tokens = all_tokens + filtered_reviewText + filtered_summary
    date_token.append([dftoken["reviewTime"][row],filtered_reviewText + filtered_summary])
uni_voc = list(set(all_tokens))

Find top 200 meaningful bigrams by using PMI measure 

In [13]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(all_tokens)
best_bigram = finder.nbest(bigram_measures.pmi, 200)

## Step 8 Combine unigrams and bigrams,then sort alphabetically  <a class="anchor" id="task8"></a>

In [14]:
# combine and convert to multi-word expressions.
for bigram in best_bigram:
    uni_voc.append(bigram)

mwe_tokenizer = MWETokenizer(uni_voc)
mwe_tokens = mwe_tokenizer.tokenize(all_tokens)

In [15]:
#sort alphabetically
mwe_tokens = list(set(mwe_tokens))
mwe_tokens = sorted(mwe_tokens, key=str.lower)
# give ID number
vocab_id = {}
for i in range(len(mwe_tokens)):
    vocab_id[mwe_tokens[i]] = i

with open("32073119_vocab.txt","a") as f:
    for word, count in vocab_id.items():
        f.write("{words}:{id}\n".format(words=word,id=count ))
f.close()

In [16]:
#aggregate texts by date
date_dictionary = {}
for date in unique_date:
    date_dictionary[date] = ''

for i in range(len(date_token)):
    string= ' '.join([str(item) for item in date_token[i][1]])
    date_dictionary[date_token[i][0]] = date_dictionary[date_token[i][0]] + string + ' '

## Step 9 create countvectorizer  <a class="anchor" id="task9"></a>

In [17]:
data_features = {}
#create countvectorizer
vectorizer = CountVectorizer(analyzer = "word")
for date in unique_date:
    data_features[date] = vectorizer.fit_transform([date_dictionary[date]])

In [18]:
vocab2 = vectorizer.get_feature_names()
with open("32073119_countVec.txt","a") as f:
    for i in range(len(unique_date)):
        message = '{}'.format(unique_date[i])
        for word, count in zip(mwe_tokens, data_features[unique_date[i]].toarray()[0]):
            if count > 0:
                w = ",{words}:{count}".format(words=vocab_id[word],count=count )
                message = message + w
        f.write(message)
        f.write("\n")
f.close()